In [1]:
from langchain_core.documents import Document 
documents=[
    Document(
        page_content="Dogs are great companion, known for thei true loyalty and friendliness.",
        metadata={"source":"mammmle-pets-doc"},
    ),
    Document(
        page_content="cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammmle-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care",
        metadata={"source":"fish-pets-doc"},
    ),
    Document(
        page_content="parrots are intelligent birds capable of mimicking human speech",
        metadata={"source":"bird-pets-doc"},
    ),
       Document(
        page_content="rabbits are social animals that need plenty of space to hop around",
        metadata={"source":"mammal-pets-doc"},
    ),
]
documents

[Document(metadata={'source': 'mammmle-pets-doc'}, page_content='Dogs are great companion, known for thei true loyalty and friendliness.'),
 Document(metadata={'source': 'mammmle-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='parrots are intelligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='rabbits are social animals that need plenty of space to hop around')]

In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B87541CF50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B87541D9A0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


d:\GenAI\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.vectorstores import FAISS
vectorstore=FAISS.from_documents(documents,embedding=embeddings)
vectorstore


In [5]:
vectorstore.similarity_search("cat")

[Document(id='20887473-fa13-45c0-a702-6fe9d5f1a444', metadata={'source': 'mammmle-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(id='8537c2a6-dece-4276-abdd-fcf441017ed6', metadata={'source': 'mammal-pets-doc'}, page_content='rabbits are social animals that need plenty of space to hop around'),
 Document(id='20e0376b-09e6-4999-af94-53d143c640d9', metadata={'source': 'mammmle-pets-doc'}, page_content='Dogs are great companion, known for thei true loyalty and friendliness.'),
 Document(id='0f74e06b-7591-430c-b28a-b98142399bc8', metadata={'source': 'bird-pets-doc'}, page_content='parrots are intelligent birds capable of mimicking human speech')]

Retrievers

In [6]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='20887473-fa13-45c0-a702-6fe9d5f1a444', metadata={'source': 'mammmle-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(id='20e0376b-09e6-4999-af94-53d143c640d9', metadata={'source': 'mammmle-pets-doc'}, page_content='Dogs are great companion, known for thei true loyalty and friendliness.')]]

In [7]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='20887473-fa13-45c0-a702-6fe9d5f1a444', metadata={'source': 'mammmle-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(id='20e0376b-09e6-4999-af94-53d143c640d9', metadata={'source': 'mammmle-pets-doc'}, page_content='Dogs are great companion, known for thei true loyalty and friendliness.')]]

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provided context only.
{question}
context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me about dogs")
print(response.content)

According to the context, dogs are great companions, known for their true loyalty and friendliness.
